<a href="https://colab.research.google.com/github/yuvrajssoni/AER850-Project3/blob/main/Step_2_3_YOLO_Training_and_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Step 2: YOLOv11 Model Training

# Install Ultralytics YOLO package
!pip install ultralytics

from ultralytics import YOLO
import shutil

# Using YOLOv11
pcb_yolo_model = YOLO("yolo11l.pt")

# Train the YOLO model on the PCB dataset
training_results = pcb_yolo_model.train(
    data="/content/drive/MyDrive/AER850 Project 3 Files/Project 3 Data/data/data.yaml",
    epochs=190,
    batch=2,
    imgsz=1280,
    optimizer="AdamW",
    lr0=5e-4,
    cos_lr=True,
    weight_decay=0.0005,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    translate=0.1,
    scale=0.5,
    mosaic=0.4,
    patience=60,
    project="/content/drive/MyDrive/AER850 Project 3 Files/Results",
    name="PCB Component Detector"
)

print("Training finished.")

# Path to BEST model generated by Ultralytics
best_model_path = (
    "/content/drive/MyDrive/AER850 Project 3 Files/Results/"
    "PCB Component Detector/weights/best.pt"
)

final_best_save_path = (
    "/content/drive/MyDrive/AER850 Project 3 Files/Results/"
    "PCB Component Detector.pt"
)

shutil.copy(best_model_path, final_best_save_path)

print("BEST model saved to:", final_best_save_path)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.3 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/AER850 Project 3 Files/Project 3 Data/data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=190, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.

In [ ]:
"Step 3: YOLOv11 Evaluation on PCB Boards"

from ultralytics import YOLO
import os
import shutil

# Load the trained PCB component detector model
trained_weights_path = (
    "/content/drive/MyDrive/AER850 Project 3 Files/Results/PCB Component Detector.pt"
)
pcb_detector_model = YOLO(trained_weights_path)

# Define paths to evaluation images
mega_board_image_path = (
    "/content/drive/MyDrive/AER850 Project 3 Files/Project 3 Data/data/evaluation/ardmega.jpg"
)
uno_board_image_path = (
    "/content/drive/MyDrive/AER850 Project 3 Files/Project 3 Data/data/evaluation/arduno.jpg"
)
raspberry_pi_board_image_path = (
    "/content/drive/MyDrive/AER850 Project 3 Files/Project 3 Data/data/evaluation/rasppi.jpg"
)

# Root results folder in Google Drive
results_root = "/content/drive/MyDrive/AER850 Project 3 Files/Results"

# Temporary YOLO output folder
temp_run_name = "temp_eval_run"
temp_output_dir = os.path.join(results_root, temp_run_name)
os.makedirs(temp_output_dir, exist_ok=True)

# Run predictions
pcb_detector_model.predict(
    source=[mega_board_image_path, uno_board_image_path, raspberry_pi_board_image_path],
    save=True,
    imgsz=928,
    conf=0.15,
    iou=0.5,
    max_det=3000,
    line_width=3,
    project=results_root,
    name=temp_run_name,
    exist_ok=True
)

archive_dir = os.path.join(results_root, "archive")
os.makedirs(archive_dir, exist_ok=True)

rename_map = {
    "ardmega.jpg": "Arduino Mega_PCB Detection.jpg",
    "arduno.jpg": "Arduino UNO_PCB Detection.jpg",
    "rasppi.jpg": "Raspberry Pi_PCB Detection.jpg",
}

for old_name, new_name in rename_map.items():
    old_path = os.path.join(temp_output_dir, old_name)
    archive_path = os.path.join(archive_dir, old_name)
    new_path = os.path.join(results_root, new_name)

    if os.path.exists(old_path):
        shutil.move(old_path, archive_path)
        shutil.copy(archive_path, new_path)

shutil.rmtree(temp_output_dir, ignore_errors=True)

print("Evaluation complete.")
print("Images saved in:")
print(results_root)



0: 928x928 1 Button, 12 Capacitors, 10 Connectors, 2 Electrolytic Capacitors, 7 ICs, 15 Resistors, 8.3ms
1: 928x928 1 Button, 11 Capacitors, 9 Connectors, 2 Diodes, 4 Electrolytic Capacitors, 4 ICs, 3 Leds, 1 Pads, 11 Resistors, 1 Transistor, 8.3ms
2: 928x928 26 Capacitors, 11 Connectors, 1 Electrolytic Capacitor, 13 ICs, 1 Inductor, 9 Leds, 26 Resistors, 8.3ms
Speed: 6.1ms preprocess, 8.3ms inference, 1.1ms postprocess per image at shape (1, 3, 928, 928)
Results saved to /content/drive/MyDrive/AER850 Project 3 Files/Results/temp_eval_run
Evaluation complete.
Images saved in:
/content/drive/MyDrive/AER850 Project 3 Files/Results
